In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime

# تحميل البيانات
data = pd.read_csv("emergency_data_2024_clean.csv")

# تجهيز البيانات
event_mapping = {value: idx for idx, value in enumerate(data["Event"].unique())}
weather_mapping = {value: idx for idx, value in enumerate(data["Weather Condition"].unique())}
disease_mapping = {value: idx for idx, value in enumerate(data["Disease Type"].unique())}

data["Event"] = data["Event"].map(event_mapping)
data["Weather Condition"] = data["Weather Condition"].map(weather_mapping)
data["Disease Type"] = data["Disease Type"].map(disease_mapping)

X = data[["Temperature (°C)", "Event", "Weather Condition", "Disease Type"]]
y = data["Patients"]

# تدريب نموذج Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# إدخال القيم من المستخدم
print("أدخل القيم التالية للتنبؤ:")
date_input = input("📅 أدخل التاريخ (YYYY-MM-DD): ")
temperature = float(input("🌡️ أدخل درجة الحرارة (°C): "))
event = input(f"🎉 اختر المناسبة ({', '.join(event_mapping.keys())}): ")
weather = input(f"☁️ اختر حالة الطقس ({', '.join(weather_mapping.keys())}): ")
disease = input(f"🩺 اختر نوع المرض ({', '.join(disease_mapping.keys())}): ")

# تحويل المدخلات
input_data = pd.DataFrame({
    "Temperature (°C)": [temperature],
    "Event": [event_mapping[event]],
    "Weather Condition": [weather_mapping[weather]],
    "Disease Type": [disease_mapping[disease]]
})

# التنبؤ باستخدام النموذج
prediction = model.predict(input_data)

# استخراج اسم اليوم من التاريخ
date_object = datetime.strptime(date_input, '%Y-%m-%d')
day_name = date_object.strftime('%A')

# عرض النتائج
print("\n--- النتائج ---")
print(f"📅 التاريخ: {date_input} ({day_name})")
print(f"👨‍⚕️ عدد المرضى المتوقعين: {int(prediction[0])} مريض")

# رسم بياني لعرض العلاقة بين درجة الحرارة وعدد المرضى
plt.figure(figsize=(10, 6))
plt.scatter(data["Temperature (°C)"], data["Patients"], alpha=0.5, label="البيانات التاريخية")
plt.scatter([temperature], [prediction[0]], color="red", label="التوقع", s=100)
plt.title("العلاقة بين درجة الحرارة وعدد المرضى", fontsize=16)
plt.xlabel("درجة الحرارة (°C)", fontsize=14)
plt.ylabel("عدد المرضى", fontsize=14)
plt.legend()
plt.grid(True)
plt.show()


أدخل القيم التالية للتنبؤ:


In [ ]:
# رسم بياني ديناميكي لعدد المرضى المتوقعين على مدار الأيام
import matplotlib.pyplot as plt

days = list(range(1, 366))
predictions = []

for day in days:
    temp = 25 + (day % 20 - 10)
    ev = list(event_mapping.keys())[day % len(event_mapping)]
    we = list(weather_mapping.keys())[day % len(weather_mapping)]
    di = list(disease_mapping.keys())[day % len(disease_mapping)]
    
    input_data_dynamic = pd.DataFrame({
        "Temperature (°C)": [temp],
        "Event": [event_mapping[ev]],
        "Weather Condition": [weather_mapping[we]],
        "Disease Type": [disease_mapping[di]]
    })
    pred = model.predict(input_data_dynamic)
    predictions.append(pred[0])

# عرض الرسم
plt.figure(figsize=(12, 6))
plt.plot(days, predictions, label="عدد المرضى المتوقع")
plt.axvline(day_of_year, color='r', linestyle='--', label="اليوم الحالي")
plt.title("عدد المرضى المتوقع بناءً على التاريخ", fontsize=16)
plt.xlabel("اليوم في السنة", fontsize=14)
plt.ylabel("عدد المرضى", fontsize=14)
plt.legend()
plt.grid(True)
st.pyplot(plt)
